In [24]:
import Control.Arrow (first)
import Data.Text (pack)
import Control.Monad

:l Plotting.hs
:l ../src/Control/Monad/Bayes/Class.hs
:l ../src/Control/Monad/Bayes/Enumerator.hs
:l ../src/Control/Monad/Bayes/Sampler.hs


# Introduction to Monad-Bayes

This serves as an interactive alternative to [the user guide](https://monad-bayes.netlify.app/probprog.html). This isn't intended as a tutorial to Haskell, but if you're familiar with probabilistic programming, the general flow of the code should look familiar.

To get a sense of how probabilistic programming with monad-bayes works, consider the following:

In [25]:
model :: MonadInfer m => Double -> m Double
model observation = do
    mean <- uniformD [-1, 0, 1]
    factor (normalPdf mean 1 observation)
    return mean

The idea of monad-bayes, and probabilistic programming languages in general is to define distributions as programs. `model` corresponds to the distribution that you would express mathematically as:

$$ P(m | o) = \frac{P(m)P(o|m)}{P(o)} = \frac{1/3 * \mathbb{N}(o; m, 1)}{\sum_{m' \in \{-1,0,1\}} 1/3 * \mathbb{N}(o; m', 1) } $$

As a program, you can think of `model` as doing the following:

- first draw from the prior over possible values of `mean` (that's the line `mean <- uniformD [-1, 0, 1]`)
- then score a draw higher according to the likelihood placed on `observation` (the argument to `model`) by a normal with $\mu$=`mean`

- then return the `mean`

To orient you on the relationship between the mathematical view of a distribution and the programming one, here are some notes:

- a distribution over values of type `a` has type `MonadInfer m => m a`
- a joint distribution over values of types `a` and `b` is a distribution over a tuple: `MonadInfer m => m (a, b)`
- a conditional distribution over values of type `a` conditioned on values of type `b` is a function into a distribution: 
    `MonadInfer m => b -> m a`



For example, if the value observed is $0.3$, then we can calculate the distribution over the mean:



In [35]:
inferredDistribution = enumerate $ model 0.3
inferredDistribution

[(-1.0,0.198111610867497),(0.0,0.44090549839518783),(1.0,0.36098289073731515)]

In [36]:
plotVega (first (pack . show) <$> inferredDistribution)

To produce this plot, we performed *inference*, to obtain the exact form of the distribution represented by `model`. Because the only random variable in `model` had a support that was small and discrete (the set $\{-1, 0, 1\}$), performing this inference exactly was straightforward.

`enumerate` is the exact inference method offered by monad-bayes.

You are encouraged to change `model` in a number of ways and observe how the results change:
- try changing the prior (currently `uniformD [-1, 0, 1]`)
- try changing the score (currently `factor (normalPdf mean 1 observation)`)
- try changing the types of the observation and latent variable (i.e. `mean`)

If you are familiar with Haskell, then it should be clear that the class of distributions you can express in this way is very broad, since we have monadic control flow. For example, you could build:

In [53]:
lengthDist :: MonadInfer m => Double -> m Int
lengthDist observation = do
    means <- replicateM 3 (model observation)
    return (length $ filter (>=1) means)

Given an observation, this is the probability on how many (out of 3) independent draws from the posterior of the model conditioned on the observation will be greater or equal to 1. Consider the hassle of defining this with an equation, and you'll see why probabilistic programming is appealing as a way of accelerating modelling and inference.

In [61]:
enumerate $ lengthDist 0.5

[(0,0.1927812120128709),(1,0.42280307652209714),(2,0.30909381616263754),(3,7.532189530239394e-2)]

In [62]:

plotVega $ fmap (first (pack . show)) $ enumerate $ lengthDist 0.99

In [63]:
plotVega $ fmap (first (pack . show)) $ enumerate $ lengthDist 0.001

Exact sampling is pretty limited. For models with continuous random variables, or large discrete ones, it is a no-go. 

The broader goal is to be able to define your distribution of interest, like `model`, and then apply different inference technique, usually approximate, to it. This is what monad-bayes (and other probabilistic programming languages) enable. See the following tutorials for details.